In [1]:
import pandas as pd
import numpy as np
import phitech.helpers.sierra as pth_sierra
import matplotlib.pyplot as plt
from progiter import ProgIter
import time

In [2]:
pth_sierra.bento_to_sierra(
    input_filepath='./mes-bento-small.mbo.dbn.zst',
    output_scid_file='/home/darchitect/wine-bottles/SierraChart/drive_c/SierraChart/Data/bento-small.scid',
    output_depth_file='/home/darchitect/wine-bottles/SierraChart/drive_c/SierraChart/Data/MarketDepthData/bento-small.2024-06-20.depth'
)

running -> bento to .scid (TODO: implement appending logic to the same scid file))
load file
convert to df


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7b258d241290>>
Traceback (most recent call last):
  File "/home/darchitect/miniconda3/envs/trading/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 
Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7b258d241290>>
Traceback (most recent call last):
  File "/home/darchitect/miniconda3/envs/trading/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 

KeyboardInterrupt

